In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture as GMM
import matplotlib.colors as mcolors
from matplotlib.patches import Ellipse

def draw_ellipse(position, covariance, ax=None, **kwargs):
    ax = ax or plt.gca()
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height,
                             angle, **kwargs))

with open('input.json', 'r') as inputFile:
    datas = json.load(inputFile)
    datas = {key:np.array(data) for key, data in datas.items()}
x1, y1 = datas['x1'], datas['y1']
x2, y2 = datas['x2'], datas['y2']
x3, y3 = datas['x3'], datas['y3']

In [3]:
import matplotlib as mpl
from sklearn.mixture import GaussianMixture as GMM
colors = list(mpl.colormaps.values())
class GMM_predictor(object):
    def __init__(self, x: np.ndarray, y: np.ndarray):
        self.x = x
        self.y = y
        self.classes = np.unique(self.y)
        self.num_class = len(self.classes)
        self.colors = colors[:self.num_class]
        self.gmm_ = None
    def draw_points(self, x, y, title=''):
        plt.figure()
        plt.xlabel('$x_1$')
        plt.ylabel('$x_2$')
        if title != '':
            plt.title(title)
        classes = np.unique(y)
        for c_idx, c in enumerate(classes):
            target = x[y == c,:]
            x1 = target[:, 0]
            x2 = target[:, 1]
            plt.scatter(x1, x2, cmap=self.colors[c_idx], s=15, label="Class {}".format(c))


        w_factor = 0.2 / self.gmm_.weights_.max()
        for c_idx, center in enumerate(self.gmm_.means_):
            plt.scatter(center[0], center[1], marker='x', s=50)

        # print(self.gmm_.covariances_.shape)
        for c_idx, (pos, covar, w) in enumerate(zip(self.gmm_.means_, self.gmm_.covariances_, self.gmm_.weights_)):
            draw_ellipse(pos, covar, alpha=w * w_factor, color='#e377c2')

        plt.legend()
        plt.show()
    def gmm_predict(self, max_iter:int=100, covariance_type:float='full', tol:float=1e-3):
        self.gmm_ = GMM(n_components=self.num_class,
                    covariance_type=covariance_type,
                    max_iter=max_iter, 
                    tol=tol)
        self.gmm_ = self.gmm_.fit(self.x)
        predict_y = self.gmm_.predict(self.x)
        return predict_y
        # self.draw_points(self.x , predict_y, 'GMM Result')


gmm = GMM_predictor(x3, y3)
# gmm.draw_points(x3, y3, 'Origin')
gmm.gmm_predict(max_iter=10)
# gmm = GMM_predictor(x2, y2)
# gmm.draw_points()
# gmm = GMM_predictor(x3, y3)
# gmm.draw_points()

array([7, 2, 2, 2, 5, 0, 0, 3, 6, 5, 0, 4, 7, 4, 4, 6, 5, 5, 6, 4, 5, 6,
       4, 2, 6, 6, 5, 2, 3, 0, 2, 0, 1, 1, 2, 5, 2, 0, 1, 1, 1, 1, 5, 3,
       4, 6, 0, 3, 4, 1, 3, 1, 4, 5, 3, 4, 0, 3, 0, 0, 0, 3, 5, 6, 3, 0,
       3, 1, 2, 2, 4, 5, 2, 4, 2, 5, 4, 7, 4, 4, 6, 7, 3, 2, 1, 3, 5, 5,
       5, 3, 2, 7, 6, 3, 2, 0, 6, 1, 1, 3, 7, 5, 6, 6, 1, 7, 5, 4, 6, 6,
       5, 5, 2, 1, 1, 3, 6, 6, 0, 0, 0, 4, 7, 7, 2, 4, 0, 7, 0, 7, 4, 6,
       5, 5, 5, 7, 5, 7, 4, 3, 5, 6, 4, 0, 1, 1, 5, 3, 2, 2, 5, 3, 1, 7,
       3, 4, 6, 3, 4, 1, 4, 5, 6, 2, 0, 2, 2, 4, 1, 7, 0, 0, 3, 6, 0, 4,
       4, 0, 3, 4, 5, 1, 3, 4, 2, 3, 0, 7, 0, 1, 5, 1, 1, 0, 7, 2, 4, 7,
       3, 2, 0, 1, 7, 3, 5, 0, 6, 7, 0, 4, 5, 2, 0, 6, 0, 4, 6, 4, 5, 6,
       0, 0, 5, 5, 3, 1, 6, 7, 7, 2, 7, 4, 5, 7, 5, 2, 0, 6, 3, 3, 0, 1,
       3, 1, 5, 0, 2, 3, 0, 7, 3, 6, 7, 4, 1, 4, 2, 3, 6, 0, 6, 6, 7, 6,
       6, 2, 5, 2, 1, 0, 4, 1, 3, 4, 0, 4, 7, 7, 1, 1, 1, 3, 4, 0, 5, 6,
       3, 1, 6, 0, 5, 3, 6, 3, 1, 6, 3, 1, 2, 0, 2,